In [2]:
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
from collections import OrderedDict
import glob
# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions

FACIAL_LANDMARKS_IDXS = OrderedDict([("mouth", (48, 68)),("right_eyebrow", (17, 22)),("left_eyebrow", (22, 27)),("right_eye", (36, 42)),    ("left_eye", (42, 48)),("nose", (27, 35)),("jaw", (0, 17))])
def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
 
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)
def shape_to_np(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = np.zeros((68, 2), dtype=dtype)
 
	# loop over the 68 facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
	# return the list of (x, y)-coordinates
	return coords





In [6]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/Users/kumaramanivelramanathan/Downloads/Emo_Detection/shape_predictor_68_face_landmarks.dat')
flag=0


In [7]:
import pandas as pd
df=pd.DataFrame(columns=['mouth_curve','mouth_length','mouth_height','eyebrow_l','eyebrow_r','eye_length','eye_height','label'])


In [8]:
df

,mouth_curve,mouth_length,mouth_height,eyebrow_l,eyebrow_r,eye_length,eye_height,label


In [9]:
for filename in glob.iglob('/Users/kumaramanivelramanathan/Downloads/Emo_Detection/Dataset/test/surprise/*'):
    image=cv2.imread(filename)
#image = cv2.imread(filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#gray = clahe.apply(image)
    gray = cv2.equalizeHist(image)
    #gray=cv2.resize(gray,(256,256))
    gray=cv2.resize(gray,(64,64))
#print(image,filename)
#gray=image
#image = imutils.resize(image, width=500)
    rects = detector(gray, 1)
    check=0
    seg=[]

# loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
	# convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        #full_face=image[shape[20][1]-10:shape[9][1],shape[1][0]:shape[16][0]+5]
        eyes=image[shape[38][1]-5:shape[42][1]+5,shape[37][0]-5:shape[40][0]+5]
        mouth=image[shape[52][1]-5:shape[58][1]+5,shape[49][0]-5:shape[55][0]+5]
        eyebrow=image[shape[20][1]-5:shape[18][1]+5,shape[18][0]-5:shape[22][0]+5]
        '''if(eyes.any() and mouth.any()):
            eyes=cv2.resize(eyes,(28,28))
            mouth=cv2.resize(mouth,(28,28))
            eyes=np.asarray(eyes)
            mouth=np.asarray(mouth)
            eyes=eyes.reshape(-1)
            mouth=mouth.reshape(-1)'''
        #eyebrow=cv2.resize(eyebrow,(64,64))
        #full_face=cv2.resize(full_face,(64,64))
        #cv2.imwrite('D:/Happy%04i.jpg'%flag,full_face)
        mouth_curve=(shape[49][1]-shape[52][1])/(shape[49][0]-shape[52][0])
        mouth_length=shape[55][0]-shape[49][0]
        mouth_height=shape[52][1]-shape[58][1]
        eyebrow_l=(shape[22][1]-shape[18][1])/(shape[22][0]-shape[18][0])
        eyebrow_r=(shape[27][1]-shape[23][1])/(shape[27][0]-shape[23][0])
        eye_length=shape[40][0]-shape[37][0]
        eye_height=shape[38][1]-shape[42][1]
        label=7
        dff=[mouth_curve,mouth_length,mouth_height,eyebrow_l,eyebrow_r,eye_length,eye_height,label]
        df.loc[len(df)] = dff


In [10]:
df

,mouth_curve,mouth_length,mouth_height,eyebrow_l,eyebrow_r,eye_length,eye_height,label
0,-0.100000,9.0,-10.0,0.153846,-0.666667,4.0,-6.0,7.0
1,-0.111111,13.0,-11.0,0.050000,-1.000000,4.0,-2.0,7.0
2,-0.333333,13.0,-17.0,-0.045455,-0.875000,4.0,-3.0,7.0
3,-0.333333,10.0,-7.0,0.000000,-0.800000,3.0,-4.0,7.0
4,-0.000000,12.0,-5.0,0.111111,-1.250000,3.0,-3.0,7.0
...,...,...,...,...,...,...,...,...
480,-0.250000,9.0,-10.0,0.181818,-1.333333,2.0,-2.0,7.0
481,-0.300000,15.0,-14.0,-0.125000,-1.000000,4.0,-2.0,7.0
482,-0.250000,12.0,-8.0,-0.083333,-0.888889,3.0,-2.0,7.0
483,-0.200000,14.0,-18.0,0.080000,-0.777778,3.0,-2.0,7.0


In [12]:
df.to_csv('/Users/kumaramanivelramanathan/Downloads/Emo_Detection/Dataset/test/suprise.csv')